## Test function sileroVADcopy.py

In [1]:
from audio_processing.sileroVADcopy import read_audio_vad

In [2]:
x = read_audio_vad(path='/home4/tuanlha/EXpressiveTTS/dataRawProcess/00_standardization/HaveASip/02 Sao nên ngừng 3s trước câu： ＂Bạn có khoẻ không？＂ - NSND Công Lý ｜ #HaveASip 122 .wav')

In [9]:
x.shape

torch.Size([37528276])

## Test processed 

In [1]:
SAMPLING_RATE = 16000

import torch
torch.set_num_threads(1)
from IPython.display import Audio
from pprint import pprint
import glob
import os
import librosa
import soundfile as sf
import torchaudio
import argparse
import csv
import json
import pandas as pd
from audio_processing.audio_utils import standardize_audio, save_sub_audio
from utils.logger import Logger
from utils.tool import check_exists
import pickle
from concurrent.futures import ThreadPoolExecutor, as_completed

Initialize logger for main


In [2]:
import os
import pickle
import csv
from multiprocessing import Pool, cpu_count

def process_episode(args):
    playlist_name, episode, folder_path = args
    episode_path = os.path.join(folder_path, playlist_name, episode)
    # print(episode_path)
    if not episode_path.endswith('.pkl'):
        episode_path += '.pkl'
    results = []
    wav_name = episode.replace('.pkl', '')  # hoặc sửa lại nếu cần lấy tên file wav khác
    try:
        with open(episode_path, 'rb') as f:
            data_clean = pickle.load(f)
        for data_point in data_clean:
            for segment in data_point[1]:
                seg_start = segment['start'] * 3 / 2
                seg_end = segment['end'] * 3 / 2
                duration = (seg_end - seg_start) / 24000
                results.append({
                    'playlist_name': playlist_name,
                    'episode_name': episode.replace('.pkl', ''),
                    'wav_name': wav_name,
                    'duration': duration
                })
    except Exception as e:
        print(f"Error processing {episode_path}: {e}")
    return results

def create_sub_vad_ver_0_parallel(folder_path='./02_vad_extract_pkl', output_csv='output.csv'):
    args_list = []
    for playlist_name in os.listdir(folder_path):
        playlist_path = os.path.join(folder_path, playlist_name)
        if not os.path.isdir(playlist_path):
            continue
        for episode in os.listdir(playlist_path):
            if episode.endswith('.pkl'):
                args_list.append((playlist_name, episode, folder_path))

    with Pool(min(cpu_count(),12)) as pool:
        all_results = pool.map(process_episode, args_list)

    # Flatten the results
    flat_results = [item for sublist in all_results for item in sublist]

    # Write to CSV
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['playlist_name', 'episode_name', 'wav_name', 'duration'])
        writer.writeheader()
        for row in flat_results:
            writer.writerow(row)

# Gọi hàm:
create_sub_vad_ver_0_parallel()


In [8]:
import os
import pickle
import csv
from multiprocessing import Pool, cpu_count

def process_episode_ver1(args):
    playlist_name, episode, folder_path = args
    episode_path = os.path.join(folder_path, playlist_name, episode)
    # print(episode_path)
    if not episode_path.endswith('.pkl'):
        episode_path += '.pkl'
    results = []
    wav_name = episode.replace('.pkl', '')  # hoặc sửa lại nếu cần lấy tên file wav khác
    try:
        with open(episode_path, 'rb') as f:
            data_clean = pickle.load(f)
        
        queue = []
        for data_point in data_clean:
            queue = []
            for segment in data_point[1]:
                if not queue:
                    queue.append(segment)
                    continue

                if (segment['start'] * 3 / 2 - queue[-1]['end'] * 3 / 2)/24000 <= 0.5:
                    queue[-1]['end'] = segment['end']
                else:
                    queue.append(segment)
            for segment in queue:
                seg_start = segment['start'] * 3 / 2
                seg_end = segment['end'] * 3 / 2
                duration = (seg_end - seg_start) / 24000
                results.append({
                    'playlist_name': playlist_name,
                    'episode_name': episode.replace('.pkl', ''),
                    'wav_name': wav_name,
                    'duration': duration
                })

    except Exception as e:
        print(f"Error processing {episode_path}: {e}")
    return results

def create_sub_vad_ver_1_parallel(folder_path='./02_vad_extract_pkl', output_csv='output1.csv'):
    args_list = []
    for playlist_name in os.listdir(folder_path):
        playlist_path = os.path.join(folder_path, playlist_name)
        if not os.path.isdir(playlist_path):
            continue
        for episode in os.listdir(playlist_path):
            if episode.endswith('.pkl'):
                args_list.append((playlist_name, episode, folder_path))

    with Pool(min(cpu_count(),8)) as pool:
        all_results = pool.map(process_episode_ver1, args_list)

    # Flatten the results
    flat_results = [item for sublist in all_results for item in sublist]

    # Write to CSV
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['playlist_name', 'episode_name', 'wav_name', 'duration'])
        writer.writeheader()
        for row in flat_results:
            writer.writerow(row)

# Gọi hàm:
create_sub_vad_ver_1_parallel()


In [2]:
path = '/home4/tuanlha/EXpressiveTTS/dataRawProcess/02_vad_extract_pkl/TunaHoatHinh/Kiếm Tiền Thời Đi Học ｜ Monsieur Tuna.pkl'
import pickle
with open(path, 'rb') as f:
    data = pickle.load(f)
data

[[[0.03096875, 1.5834687500000002, 'SPEAKER_00', True, True],
  [{'start': 1568, 'end': 24840}],
  [{'start': 1568, 'end': 24840}]],
 [[1.9209687500000001, 35.33346875, 'SPEAKER_00', True, True],
  [{'start': 1056, 'end': 20448},
   {'start': 22560, 'end': 57824},
   {'start': 59936, 'end': 109536},
   {'start': 116768, 'end': 170976},
   {'start': 177696, 'end': 283104},
   {'start': 287776, 'end': 380384},
   {'start': 381984, 'end': 464864},
   {'start': 469024, 'end': 526816},
   {'start': 528416, 'end': 534600}],
  [{'start': 1056, 'end': 20448},
   {'start': 22560, 'end': 30688},
   {'start': 37920, 'end': 57824},
   {'start': 59936, 'end': 69088},
   {'start': 71712, 'end': 109536},
   {'start': 116768, 'end': 170976},
   {'start': 177696, 'end': 185824},
   {'start': 187936, 'end': 283104},
   {'start': 287776, 'end': 380384},
   {'start': 381984, 'end': 464864},
   {'start': 469024, 'end': 526816},
   {'start': 528416, 'end': 534600}]],
 [[36.04221875, 104.13284375, 'SPEAKER_0

# Phân tích duration khi thi triển theo 2 cách

In [10]:
import pandas as pd
# Đọc dữ liệu từ hai file CSV
data_ver_0 = pd.read_csv('output.csv')
data_ver_1 = pd.read_csv('output1.csv')

# Hàm tính toán thống kê
def calculate_statistics_from_csv(data):
    durations = data['duration']
    total_duration = durations.sum()
    num_files = len(durations)
    avg_duration = durations.mean()
    std_dev_duration = durations.std()

    return {
        'total_duration': total_duration,
        'num_files': num_files,
        'avg_duration': avg_duration,
        'std_dev_duration': std_dev_duration
    }

# Tính toán thống kê cho từng cách
statistics_ver_0 = calculate_statistics_from_csv(data_ver_0)
statistics_ver_1 = calculate_statistics_from_csv(data_ver_1)

# In kết quả
print("Thống kê cho create_sub_vad_ver_0_parallel:")
print(f"Tổng thời gian: {statistics_ver_0['total_duration']} giây")
print(f"Số lượng file: {statistics_ver_0['num_files']}")
print(f"Thời gian trung bình: {statistics_ver_0['avg_duration']} giây")
print(f"Độ lệch chuẩn thời gian: {statistics_ver_0['std_dev_duration']} giây")

print("\nThống kê cho create_sub_vad_ver_1_parallel:")
print(f"Tổng thời gian: {statistics_ver_1['total_duration']} giây")
print(f"Số lượng file: {statistics_ver_1['num_files']}")
print(f"Thời gian trung bình: {statistics_ver_1['avg_duration']} giây")
print(f"Độ lệch chuẩn thời gian: {statistics_ver_1['std_dev_duration']} giây")

Thống kê cho create_sub_vad_ver_0_parallel:
Tổng thời gian: 1378602.2958125002 giây
Số lượng file: 434258
Thời gian trung bình: 3.1746157717589547 giây
Độ lệch chuẩn thời gian: 3.93364092960002 giây

Thống kê cho create_sub_vad_ver_1_parallel:
Tổng thời gian: 1418692.4478124997 giây
Số lượng file: 247088
Thời gian trung bình: 5.741648513130948 giây
Độ lệch chuẩn thời gian: 13.78225497107309 giây


In [11]:
# Lọc dữ liệu cho nhóm 1: HaveASipClean và TroChuyenCungDinhDoanClean
group_1 = data_ver_0[data_ver_0['playlist_name'].isin(['HaveASipClean', 'TroChuyenCungDinhDoanClean'])]

# Lọc dữ liệu cho nhóm 2: Các playlist_name còn lại
group_2 = data_ver_0[~data_ver_0['playlist_name'].isin(['HaveASipClean', 'TroChuyenCungDinhDoanClean'])]

# Hàm tính toán thống kê
def calculate_statistics(data):
    durations = data['duration']
    total_duration = durations.sum()
    num_files = len(durations)
    avg_duration = durations.mean()
    std_dev_duration = durations.std()

    return {
        'total_duration': total_duration,
        'num_files': num_files,
        'avg_duration': avg_duration,
        'std_dev_duration': std_dev_duration
    }

# Tính toán thống kê cho từng nhóm
statistics_group_1 = calculate_statistics(group_1)
statistics_group_2 = calculate_statistics(group_2)

# In kết quả
print("Thống kê cho nhóm 1 (HaveASipClean và TroChuyenCungDinhDoanClean):")
print(f"Tổng thời gian: {statistics_group_1['total_duration']} giây")
print(f"Số lượng file: {statistics_group_1['num_files']}")
print(f"Thời gian trung bình: {statistics_group_1['avg_duration']} giây")
print(f"Độ lệch chuẩn thời gian: {statistics_group_1['std_dev_duration']} giây")

print("\nThống kê cho nhóm 2 (các playlist_name còn lại):")
print(f"Tổng thời gian: {statistics_group_2['total_duration']} giây")
print(f"Số lượng file: {statistics_group_2['num_files']}")
print(f"Thời gian trung bình: {statistics_group_2['avg_duration']} giây")
print(f"Độ lệch chuẩn thời gian: {statistics_group_2['std_dev_duration']} giây")

Thống kê cho nhóm 1 (HaveASipClean và TroChuyenCungDinhDoanClean):
Tổng thời gian: 298041.869375 giây
Số lượng file: 106234
Thời gian trung bình: 2.8055224257299924 giây
Độ lệch chuẩn thời gian: 2.312380838639525 giây

Thống kê cho nhóm 2 (các playlist_name còn lại):
Tổng thời gian: 1080560.4264375002 giây
Số lượng file: 328024
Thời gian trung bình: 3.2941505086136993 giây
Độ lệch chuẩn thời gian: 4.323736563418136 giây


In [12]:
# Lọc dữ liệu cho nhóm 1: HaveASipClean và TroChuyenCungDinhDoanClean
group_1 = data_ver_1[data_ver_1['playlist_name'].isin(['HaveASipClean', 'TroChuyenCungDinhDoanClean'])]

# Lọc dữ liệu cho nhóm 2: Các playlist_name còn lại
group_2 = data_ver_1[~data_ver_1['playlist_name'].isin(['HaveASipClean', 'TroChuyenCungDinhDoanClean'])]

# Hàm tính toán thống kê
def calculate_statistics(data):
    durations = data['duration']
    total_duration = durations.sum()
    num_files = len(durations)
    avg_duration = durations.mean()
    std_dev_duration = durations.std()

    return {
        'total_duration': total_duration,
        'num_files': num_files,
        'avg_duration': avg_duration,
        'std_dev_duration': std_dev_duration
    }

# Tính toán thống kê cho từng nhóm
statistics_group_1 = calculate_statistics(group_1)
statistics_group_2 = calculate_statistics(group_2)

# In kết quả
print("Thống kê cho nhóm 1 (HaveASipClean và TroChuyenCungDinhDoanClean):")
print(f"Tổng thời gian: {statistics_group_1['total_duration']} giây")
print(f"Số lượng file: {statistics_group_1['num_files']}")
print(f"Thời gian trung bình: {statistics_group_1['avg_duration']} giây")
print(f"Độ lệch chuẩn thời gian: {statistics_group_1['std_dev_duration']} giây")

print("\nThống kê cho nhóm 2 (các playlist_name còn lại):")
print(f"Tổng thời gian: {statistics_group_2['total_duration']} giây")
print(f"Số lượng file: {statistics_group_2['num_files']}")
print(f"Thời gian trung bình: {statistics_group_2['avg_duration']} giây")
print(f"Độ lệch chuẩn thời gian: {statistics_group_2['std_dev_duration']} giây")

Thống kê cho nhóm 1 (HaveASipClean và TroChuyenCungDinhDoanClean):
Tổng thời gian: 303815.53737499996 giây
Số lượng file: 78657
Thời gian trung bình: 3.8625365495124395 giây
Độ lệch chuẩn thời gian: 4.382423495631412 giây

Thống kê cho nhóm 2 (các playlist_name còn lại):
Tổng thời gian: 1114876.9104374999 giây
Số lượng file: 168431
Thời gian trung bình: 6.61919070977136 giây
Độ lệch chuẩn thời gian: 16.34838340546547 giây


# Test VAD clean

In [18]:
import pickle
# Đường dẫn đến file pickle
pickle_file_path = '/home4/tuanlha/EXpressiveTTS/dataRawProcess/02_vad_extract_pkl/HaveASipClean/Ai cũng cần những lúc không làm gì cả! - Mỹ Anh ｜ #HaveASip 160.pkl'
# Mở file pickle và đọc dữ liệu
with open(pickle_file_path, 'rb') as file:

    data = pickle.load(file)    
data

[[[0.23346875, 6.122843750000001, 'SPEAKER_01', True, True],
  [{'start': 544, 'end': 63456},
   {'start': 65056, 'end': 81376},
   {'start': 84512, 'end': 94230}],
  [{'start': 544, 'end': 63456},
   {'start': 65056, 'end': 81376},
   {'start': 84512, 'end': 94230}]],
 [[6.814718750000001, 9.008468750000002, 'SPEAKER_01', True, True],
  [{'start': 544, 'end': 35100}],
  [{'start': 544, 'end': 35100}]],
 [[10.274093750000002, 12.19784375, 'SPEAKER_01', True, True],
  [{'start': 544, 'end': 30780}],
  [{'start': 544, 'end': 30780}]],
 [[12.974093750000002, 16.129718750000002, 'SPEAKER_01', True, True],
  [{'start': 5664, 'end': 50490}],
  [{'start': 5664, 'end': 50490}]],
 [[16.652843750000002, 18.47534375, 'SPEAKER_01', True, True],
  [{'start': 1056, 'end': 29160}],
  [{'start': 1056, 'end': 29160}]],
 [[19.15034375, 22.57596875, 'SPEAKER_01', True, True],
  [{'start': 32, 'end': 32736}, {'start': 35360, 'end': 54810}],
  [{'start': 32, 'end': 32736}, {'start': 35360, 'end': 54810}]],

In [19]:
import pickle
# Đường dẫn đến file pickle
pickle_file_path = '/home4/tuanlha/EXpressiveTTS/dataRawProcess/01_clean_diarization/HaveASipClean/Ai cũng cần những lúc không làm gì cả! - Mỹ Anh ｜ #HaveASip 160.pkl'
# Mở file pickle và đọc dữ liệu
with open(pickle_file_path, 'rb') as file:

    data = pickle.load(file)    
data

[[0.23346875, 6.122843750000001, 'SPEAKER_01', True, True],
 [6.814718750000001, 9.008468750000002, 'SPEAKER_01', True, True],
 [10.274093750000002, 12.19784375, 'SPEAKER_01', True, True],
 [12.974093750000002, 16.129718750000002, 'SPEAKER_01', True, True],
 [16.652843750000002, 18.47534375, 'SPEAKER_01', True, True],
 [19.15034375, 22.57596875, 'SPEAKER_01', True, True],
 [23.04846875, 24.246593750000002, 'SPEAKER_01', True, True],
 [24.38159375, 26.980343750000003, 'SPEAKER_01', True, True],
 [28.887218750000002, 35.721593750000004, 'SPEAKER_01', True, True],
 [35.87346875, 38.42159375, 'SPEAKER_06', True, True],
 [38.759093750000005, 49.03596875, 'SPEAKER_06', True, False],
 [58.51971875, 59.835968750000006, 'SPEAKER_01', True, True],
 [60.44346875, 62.67096875, 'SPEAKER_01', True, True],
 [64.20659375000001, 65.55659375, 'SPEAKER_01', False, True],
 [67.90221875, 69.15096875, 'SPEAKER_01', False, True],
 [70.06221875, 76.71096875, 'SPEAKER_01', True, False],
 [77.48721875000001, 79

In [12]:
!pip uninstall pydrive
!pip install pydrive2

Found existing installation: PyDrive 1.3.1
Uninstalling PyDrive-1.3.1:
  Would remove:
    /home/tuanlha/miniconda3/envs/TuanLHA/lib/python3.10/site-packages/PyDrive-1.3.1.dist-info/*
    /home/tuanlha/miniconda3/envs/TuanLHA/lib/python3.10/site-packages/pydrive/*
Proceed (Y/n)? ERROR: Operation cancelled by user
^C
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 10.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: cryptography
    Found existing installation: cryptography 44.0.3
    Uninstalling cryptography-44.0.3:
      Successfully uninstalled cryptography-44.0.3


In [1]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive

gauth = GoogleAuth()


In [2]:
%cd /home4/tuanlha/EXpressiveTTS/dataRawProcess

/home4/tuanlha/EXpressiveTTS/dataRawProcess


In [3]:
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1014512554043-4ohovcmu2r6ahb7ucjr9p6f3i9ijmb7b.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8090%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=online&response_type=code

Authentication successful.


In [6]:
%cd /home4/tuanlha/EXpressiveTTS/dataRawProcess/__Final_data2

/home4/tuanlha/EXpressiveTTS/dataRawProcess/__Final_data2


In [7]:
# Nén thư mục (nếu chưa nén)
import shutil, os

for episode in os.listdir('.'):
    shutil.make_archive(episode, 'zip', episode)

    # Upload file zip
    upload_file = drive.CreateFile({'title': f'{episode}.zip'})
    upload_file.SetContentFile(f'{episode}.zip')
    upload_file.Upload()

In [23]:
upload_file.SetContentFile(f'/home4/tuanlha/EXpressiveTTS/dataRawProcess/__Final_data/SPEAKER_1.zip')

In [24]:
upload_file.Upload()